In [56]:
%matplotlib inline

# Basic Image Classifier with Radial Prediction Layer

Jupyter Notebook adapted from: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

Workflow:

1. Load and normalizing the CIFAR10 training and test datasets using
   ``torchvision``
2. Define a Convolutional Neural Network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the test data

## 0. Use GPU

In [57]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [58]:
torch.cuda.set_device(0)

## 1. Load and normalize data
The output of torchvision datasets are PILImage images of range [0, 1].
We transform them to Tensors of normalized range [-1, 1].

In [59]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## 2. Define a Convolutional Neural Network

In [60]:
import torch.nn as nn

class RadialPredictionLayer(torch.nn.Module):
    """ The RPL classification layer with fixed prototypes
    """
    def __init__(self, in_features, out_features, a = 1. , beta = 1.):
        super(RadialPredictionLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        self.a = a
        self.c = np.sqrt(2*a)
        self.c_ = self.c / 2.
        self.beta = beta
        # Initialize prototype unit vectors on each axis 
        self.prototypes = nn.Parameter(self.a * torch.diag((torch.ones(self.in_features))), requires_grad=False)


    def forward(self, x):
        # Calculate the Euclidian distance between row vectors and prototypes
        return - self.beta * torch.cdist(x,self.prototypes)
    
    def predict(self, x):
        forwarded = self.forward(x)
        distances = (-1./self.beta)* forwarded
        probabilities = np.exp(forwarded)
        # apply threshold rule
        probabilities[distances >= self.c_] = 0
        return probabilities



class RadialPredictionLoss(torch.nn.Module):
    """ Loss for networks using Radial Predition Layer
    """
    def __init__(self):
        super(RadialPredictionLoss, self).__init__()
    
    def forward(self, predictions, targets):
        batch_size = predictions.shape[0]
        loss = - predictions[range(batch_size), targets]
        return loss.sum()/batch_size 

In [61]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.rpl = RadialPredictionLayer(10,10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.rpl(x)
        return x
    
    def predict(self,x):
        return self.rpl.predict(self.forward(x))


net = Net()

In [62]:
import numpy as np
np.exp(RadialPredictionLayer(3,3).forward(torch.tensor([[0,1.,0]])))

tensor([[0.2431, 1.0000, 0.2431]])

## 3. Define a Loss function and optimizer

In [63]:
import torch.optim as optim

criterion = RadialPredictionLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## 4. Train the network
We simply have to loop over our data iterator, and feed the inputs to the
network and optimize.

In [64]:
next(iter(trainloader))[1][0]

tensor(6)

In [65]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.934
[1,  4000] loss: 0.898
[1,  6000] loss: 0.876
[1,  8000] loss: 0.846
[1, 10000] loss: 0.831
[1, 12000] loss: 0.806
[2,  2000] loss: 0.789
[2,  4000] loss: 0.778
[2,  6000] loss: 0.754
[2,  8000] loss: 0.743
[2, 10000] loss: 0.742
[2, 12000] loss: 0.730
Finished Training


Save trained model

In [66]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

## 5. Test the network on the test data

We will check this by predicting the class label that the neural network
outputs, and checking it against the ground-truth. If the prediction is
correct, we add the sample to the list of correct predictions.

Load saved model

In [67]:
net = Net()
net.load_state_dict(torch.load(PATH))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

Okay, now let us see what the neural network thinks these examples above are:



In [68]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net.predict(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 10 %


In [70]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net.predict(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane :  0 %
Accuracy of   car :  0 %
Accuracy of  bird :  0 %
Accuracy of   cat :  0 %
Accuracy of  deer :  0 %
Accuracy of   dog :  0 %
Accuracy of  frog :  0 %
Accuracy of horse :  0 %
Accuracy of  ship :  0 %
Accuracy of truck : 100 %
